In [1]:
import torch
import torch.nn.functional as torchFunc
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open("names.txt", 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [ ]:
characters = sorted(list(set(''.join(words))))
char_to_int = {s:i+1 for i,s in enumerate(characters)}
char_to_int['.'] = 0
int_to_char = {i:s for s,i in char_to_int.items()}
print(int_to_char)

Building a database to train on database.<br>
Will have context and output.<br>
For emma, possible database is (given block size is 3)
<ul>
<li>... -> e</li>
<li>..e -> m</li>
<li>.em -> m</li>
</ul>

In [24]:
block_size = 3
context_set, result_set = [], []

for word in words:
    context = [0]*block_size # '...' -> initial context

    for character in word + '.':
        result = char_to_int[character]
        context_set.append(context)
        result_set.append(result)
        
        # print(''.join(int_to_char[i] for i in context), "-->", character)

        context = context[1:] + [result]

X = torch.tensor(context_set)
Y = torch.tensor(result_set)

In [27]:
look_up = torch.randn((27, 2))
# look_up[5]
# torchFunc.one_hot(torch.tensor(5), num_classes=27).float() @ look_up

<h3>Embedding strips down dimension</h3>
In our case, we have 27 dimension, 27 possible character
We strip it to 2 dimension

After embedding calculation, we need a way to look up table of some sort, 
that can take any character and output that embedded value

In our case, look_up is that table

<h3>We can access look up in two ways</h3>
look_up[ix] to get the necessary vector
also we can do this
look_up matmul one_hot(ix)

Both results in same stuff... only this time, 
the entire look_up then also becomes a part of the neural net 
and can be used as weights of the first layer

<h3>Now actual algorithm</h3>

input is 3 element context, each is embedded to 2D vector... <br>
input is 6 dimensional<br>
output hyper parameter... we choose 100 just because<br>
bias adds to all output so 100 dimension<br>

In [30]:
Weight_1 = torch.randn((6, 100))
bias_1 = torch.randn((100))